## 测试数据库连接

In [ ]:
from pymysql import Connection

# Initialize database connection and get schema description
def init_db(host:str, port:str, user:str, password:str, database:str) -> Connection:
    return Connection(host=host, port=int(port), user=user, password=password, database=database)

def get_db_schema(conn:Connection, db_name:str):
    sql = f"show tables from {db_name}"
    with conn.cursor() as cursor:
        cursor.execute(sql)
        tables = cursor.fetchall()
        tables_schema = []
        for table in tables:
            sql = f"show create table {table[0]}"
            cursor.execute(sql)
            create_table_sql = cursor.fetchone()
            tables_schema.append(create_table_sql)
        return tables_schema

# Example usage
host = "localhost"
port = "3306"
user = "root"
password = "-"
database = "chat_mysql"

dbConnection = init_db(host, port, user, password, database)
schema = get_db_schema(dbConnection, database)
print(schema)


## 测试agent路径

In [ ]:
question = 'ABCDE菜餐饮管理有限公司的联系电话和地址分别是？'
question = '你好呀！'

import json
from openai import OpenAI
from config import *
from tool import *

client = OpenAI(
    base_url=BASE_URL_BAILIAN,
    api_key=DASHSCOPE_API_KEY,
)

system_prompt = f"""
你是一位专业的DBA，根据以下<schema>中的表结构，根据用户的问题编写SQL语句，允许使用模糊匹配 LINK操作符。
<schema> {schema} </schema>
"""

response = client.chat.completions.create(
    model="qwen2.5-14b-instruct",
    messages=[{"role":"system","content":system_prompt},{"role":"user","content":question}],
    tools=TOOLS,
    tool_choice="auto",
)

print(response.choices[0].finish_reason)

if response.choices[0].finish_reason == "tool_calls":
    for tool_call in response.choices[0].message.tool_calls:
        function_name = tool_call.function.name
        function_args = json.loads(tool_call.function.arguments)

        if function_name == "search_mysql":
            answer = search_mysql(schema,dbConnection,function_args["question"])
            print(answer)
            
# print(response)
